# Equity Real World SVJD Tool Specifcation

Our aim here is to use the functionality in the **MoodysAnalytics.Calibration.Tool.RealWorldEquity** project to replicate the functionality in the excel **CalibrationTool_Best_Views_SVJD** used in production.  

### Excel tool example

See [EndSep2017_RW_SVJD_CalibrationTool_Best_Views_SVJD.xlsm](V:\Deliverables\StandardCalibrations\Calibration\EndSep2017\Real World\EndSep2017_RW_SVJD_CalibrationTool_Best_Views_SVJD.xlsm) for an example of the current process.

### See also - Constant Volatility

The layout of this tool will be similar to the **Equity Constant Volatility** tool. It is also a *multi-asset* tool that will create at the end of the process a fit for each of the individual assets that we define as part of the tool run.  

It will also take in as a Model input the result of the **Factor Calibration** performed by that tool. This will be the "combined" version of the results, and will reqiuire the **factorLoadingsArray** parameter.  

### Validation not included

We will need to replicate the *validation* step in the current excel tool. This will be done outside of the calibration tool method, and will be its own tool, with the approach TBD.  

It is likely we can use the same tool for both SVJD and constant volatility validation, albeit with lots of additional outputs that are required for SVJD only.  

## SVJD Calibration in Ghost

Like the constant volatility tool, there will be two stages of calibration, that come from separate tool run methods in the Ghost code base.  

We will also need to use a "sorter tool" for each asset, to extract the "per-asset" result from the final "combined" results.  

Unlike the constant volatilty tool, the first tool that we use is not a complete calibration on its own - it is an intermediate step on the way to the final calibration. This could lead to us considering creating a single tool wrapper around both methods, but I would lean toward separate Mercury tools so that these can be maintained separately.  

### SVJD Calibration in a nutshell

Boiled down to its core, the steps that we need to do in the SVJD calibration process is:

+ Calibrate the *Volatility* of the **First Equity Factor** as an SVJD model.
    + This is targeted to market *one month volatility* targets.
    + We start with the existing **Assets.EQ.Factors.Sigma.Const** model for all (6) factors, which are using a *Constant Volatility* model for their volatility.
    + We make the **First Equity Factor** an SVJD model, and recalibrate.
+ Calibrate the *Volatility* of **Each Equity Asset** using an SJVD model for the *specific volatility* of the asset.
    + The assets will take a *Systematic Volatility* as the product of the **Factor Calibration** and the **Factor Loadings**
    + We will take in the *Specific Volatility* for each asset - this is the result from the constant volatility calibration.
    + The **Factor Loadings** come from a constant volatility calibration, and remain unchanged.
    + The **Jump Parameters** of the **First Equity Factor** SVJD model are calibrated to market *Skew* and *Kurtosis* targets.
    + We then **calculate** the individual SVJD model calibration for each asset.
    
#### The "gotcha" here.

There are a lot of "in the margin" calculations that sit outside of the method outputs that produce the actual model calibration for the SVJD models for the **First Equity Factor** and **Each Equity Asset**.  

These do appear to all reside in the SVJD code base, so we should not need to port anything from the excel wrapper this time, touch wood.  

### TMT Diagram

We can start this time by showing the proposed TMT diagram for the entire SVJD calibration process.  

![](images/TMT_SVJD_3ToolSolution.svg)


So, as described in the steps above.  

We should now go a little deeper into the Ghost code, and see how we map into these tools.  

## Tool Method Mapping

The methods that we need to use from the Ghost project are:

